# Importations

In [1]:
!pip install pandas numpy matplotlib seaborn

In [2]:
# Librairies I will use for these visualizations
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

import plotly.express as px
import altair as alt
import bokeh

In [180]:
# read the database
conso = pd.read_csv("conso-compo-alim-vf-mad-datagouv2021.csv", delimiter=";")

/var/folders/q1/d_r20k6n23gcyf1gb0c7j0xm0000gn/T/ipykernel_38464/1242481672.py:2: DtypeWarning: Columns (19,31,51) have mixed types. Specify dtype option on import or set low_memory=False.
  conso = pd.read_csv("conso-compo-alim-vf-mad-datagouv2021.csv", delimiter=";")


- Perso

In [186]:
conso.head()

,NOIND,occ_hdeb,occ_alim_libelle,aliment_libelle_INCA3,qte_conso,facette_06_libelle,facette_09_libelle,facette_10_libelle,aliment_marque_bio,aet,...,sodium,sel,magnesium,phosphore,potassium,calcium,fer,cuivre,zinc,iode
0,110100101,11:00,eau,eau (du robinet),147.50,non renseigné,non renseigné,non renseigné,NaN,0.0,...,2.40,0.0076,0.99,0.0,0.73,7.13,0.030,0.017,0.011,0.0
1,110100101,12:30,jus d orange,jus de fruits 100% pur jus,500.00,orange,non renseigné,préparé par un service de restauration rapide,0.0,45.0,...,4.81,0.0120,9.85,16.2,170.00,0.41,0.068,0.025,0.045,0.7
2,110100101,12:30,poulet,matière grasse de cuisson n.s.,3.22,non renseigné,non renseigné,non renseigné,NaN,0.0,...,0.00,0.0000,0.00,0.0,0.00,0.00,0.000,0.000,0.000,0.0
3,110100101,12:30,frites,frite (de pommes de terre) à la friteuse,107.00,non renseigné,non renseigné,préparé par un service de restauration rapide,0.0,267.0,...,29.00,0.0730,23.30,106.0,684.00,23.80,0.900,0.120,0.360,2.5
4,110100101,12:30,poulet,nugget de poulet,161.00,non renseigné,non renseigné,préparé par un service de restauration classique,NaN,234.0,...,614.00,1.5400,23.00,198.0,247.00,11.00,1.210,0.084,1.580,2.5


In [185]:
conso.shape

(256301, 41)

In [135]:
conso['facette_10_libelle'].unique()

array(['non renseigné', 'préparé par un service de restauration rapide',
       'préparé par un service de restauration classique',
       'produit par un industriel', 'préparé par un artisan',
       'ne connait pas le mode de préparation ou de production de l’aliment',
       'préparé maison', 'préparé par la cantine',
       'préparé par un distributeur automatique',
       'non préparé maison sans précision'], dtype=object)

In [ ]:
conso.iloc[20000, 0:51]

In [138]:
conso['NOIND'].nunique()

4114

# Data Transformation

Only 'Pop3' in the POPULATION column, and only '1' in the aliment_marque_enrichi column, so I delete these column. 

In [181]:
conso = conso.drop('POPULATION', axis=1); # ; pour pas d'output

In [182]:
conso = conso.drop('aliment_marque_enrichi', axis=1);

Many columns are not consistent for my analysis

In [183]:
columns1 = ['NUM_LIGNE', 'R24_num', 'R24_nombre', 'R24_pond', 'occ_lieu', 'aliment_code_INCA3', 'aliment_code_FX']
columns2 = ['occ_type', 'occ_alim_num', 'qte_conso_pond', 'occ_alim_num_seq', 'gpe_INCA3', 'aliment_libelle_FX']
columns3 = ['polyols', 'ag_16_0', 'ag_18_0', 'ag_14_0', 'ag_12_0', 'ag_10_0', 'ag_04_0', 'ag_06_0', 'ag_08_0']
columns4 = ['agmi', 'ag_18_1_ole', 'agpi', 'ag_18_2_lino', 'ag_18_3_a_lino', 'ag_20_4_ara', 'ag_20_5_epa', 'ag_20_6_dha']
columns5 = ['facette_01', 'facette_02', 'facette_03', 'facette_04', 'facette_05', 'facette_06', 'facette_07']
columns6 = ['facette_08', 'facette_09', 'facette_10', 'facette_12', 'facette_13', 'facette_14', 'facette_19']
columns7 = ['facette_20', 'facette_25', 'facette_27']
columns8 = ['facette_12_libelle', 'facette_13_libelle', 'facette_14_libelle', 'facette_25_libelle', 'facette_27_libelle']
columns9 = ['aesa', 'manganese']
columns10 = ['retinol', 'beta_carotene', 'selenium']
columns_to_delete = columns1 + columns2 + columns3 + columns4 + columns5 + columns6 + columns7 + columns8 + columns9 + columns10

conso = conso.drop(columns=columns_to_delete)

Some columns have more than 75% of the information missing. So I delete these columns.

In [ ]:
# Calculus formula to know how many % is missing values in the column
missing = (conso['facette_02_libelle'] == 'non renseigné').sum()
total = conso.shape[0]

percentage = (missing * 100) / total
print(percentage)

In [184]:
columns1 = ['facette_01_libelle', 'facette_02_libelle', 'facette_03_libelle', 'facette_04_libelle']
columns2 = ['facette_05_libelle', 'facette_07_libelle', 'facette_08_libelle', 'facette_19_libelle', 'facette_20_libelle']
columns_null = columns1 + columns2

conso = conso.drop(columns=columns_null)

In [187]:
conso.columns

Index(['NOIND', 'occ_hdeb', 'occ_alim_libelle', 'aliment_libelle_INCA3',
       'qte_conso', 'facette_06_libelle', 'facette_09_libelle',
       'facette_10_libelle', 'aliment_marque_bio', 'aet', 'proteines',
       'glucides', 'sucres', 'amidon', 'fibres', 'acides_organiques',
       'lipides', 'ags', 'alcool', 'eau', 'vitamine_b1', 'vitamine_b2',
       'vitamine_b3', 'vitamine_b5', 'vitamine_b6', 'vitamine_b9',
       'vitamine_b12', 'vitamine_c', 'vitamine_d', 'vitamine_e', 'vitamine_k2',
       'sodium', 'sel', 'magnesium', 'phosphore', 'potassium', 'calcium',
       'fer', 'cuivre', 'zinc', 'iode'],
      dtype='object')

# Data Visualization

Ideas: 
- 